In [1]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr-serverless connect --application-id 00fo38nsirh68609 --language python --emr-execution-role-arn arn:aws:iam::442426877041:role/service-role/AmazonEMRStudio_RuntimeRole_1731016215290

Initiating EMR Serverless connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,00fo6c5j6jc5nh0a,pyspark,idle,"<a target=""_blank"" href=""https://j-00fo6c5j6jc5nh0a.dashboard.emr-serverless.us-east-1.amazonaws.com/?authToken=eDAxDqgRJw2LSNyU3gsxQ-69jMS1Cw7wQ2JmLhRodj_mQLglNbWGJana_EhwizJRZax3ELigw1FtKRUaeDQnzAZ4Scv4ViibHpCQ6dErEQzmvK4iCPuyx5Cb7yf9a58USzSxJYokJf_F_1cGx6tdIE9QyVxBxS8iQj0AO72fX1mfiatOEaYP7p1Mr2wRyznJQNgvbYn3qOyrvj_jW_k1orJuVkwQ2mJc6y0Cf4Vq4EW8cx3HmwRe7Qwn-43kRsh4kjr3zI1rsZyptTYN1Tj54Va7fXeMQ-RdfN_5GhtlfnmFeALYeL5FwyLG28xTsjPNFNuemXccP_HqTSzambnliNjvJH2hAJIU1A8IAG6GU_OIGVOZDiDR3VwAzDNuDLgmSW4gFlNp81QSolgHERkCb2qJf9-86OI2yU_QO2gymkw0PGkonjCRTij3wVMD8aopS2zawT-EM7R2l_uNi14Gasi3jaHIXDGrycqN45z7rJ_Ow-6V1yPbBNaScQoyIxc-6EwrbwXvPpRrO3VQ8uGCfWKbjB4l79sDTcL7YozthsLpoSduDp5e5R4ro2PicwvU_HS6k0gKBWQc5kGhjOZWWbCEaG5sNKB-JcMeB8sxaEF4FM9PHMJqqbYRgpbk0e_Jx3M7gdIi51v6aDRpH_nJ9ec58g9h8PXNJ9TERjDRpPdt0929mpLc_TBAhUvykMpihl6pt6dpVbjaCj49gdI4XU_A79ZFnDfFzqJiOEMyGpVLAwAVdQxbRNc-QNOrhTUAD2byG_f1OYZo2v4HweCMoYgqyPhrlH9_jOcnHFQG56zmkFHlKjvsvT_WP8eOXi68aS_JLRNdT_MG2nTwDaonSkajJQPgPnLptpe0WKwis8X8SbKCFb8TiUjdLUC-ARrlTBQnwm2xNSy3g5MwLaKkzGbqbFkFMbetW38VwzyV0HpKi2Cxfu3Bv-rhJpOMqwF9raE.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0VQV2dxTnpvVzllc3RTa2xMQjJKbTZBQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREZNOU1DTm02WktiWGc4bFVRSUJFSUJiU0NUamlWTVlQL05mOENveVoyK1gwUkVlcTM5OVMwZGZjamNkbTgza2hyYy83NSs5bkM4YktlaTM4dU9lcGQzTENuTG53RVEvSkc2U2RiZjh5ZW5QYldVTkZmS0tsR2x6K0lTZnFJdFUyWmtzTmtaL2lkUHpSc2N0aFE9PSJ9"">Link","<a target=""_blank"" href=""https://j-00fo6c5j6jc5nh0a.dashboard.emr-serverless.us-east-1.amazonaws.com/logs/SPARK_DRIVER/stderr.gz?authToken=eDAxco5EtdG921Z_mHuNcOzdE4IBVYfjQw9LdcRupEquzA1xSpl2R7H1mbwLszbqVDPwgYM98vp2xef3b20a7Ep8pPMHMIIpZQ0P3ghnX4xTHzZF0VnkEYjb1z-9WXc7_Qe2PTjPUmr5leaOPqy1tskMgy1jZVc5ZPwe_TiGfB4pbUzHqUIh8223tkd8V5dr494-3HGHXu5upZCyKHOEwVG13vpJvB3ssYS5Zp4QuvFI9yCK4AjE10KPBvlVLkwgsHAvG6ls0F2Z5Kf1_C4QhYVijvoxIFLK-PE6s1tqcl-_55MtESdX4F-EQv4kpUcJpZSZVhQ2DzmBWDcNsGbPOx07LH6-mKMFKZINJGjoqcNTdFIYVnX3wdIoSfhxKC9PYpeSKEJM_OvsoPeGAfVSNOAK56Y6CVUJ47UYmbmwGmLSR4Ohs1bcvdR5H4124NMLxEf2XzVH0PQBZ1enGkg3-QV1C98c2-Vd4bN588bMg6nl4HBfxhj1ifUkx3iuvLTm0XVFmKri5iqR99ZI9WFo2QwR5kcgLnWBO_JtHchN7d-9RkJeEqwhF2RU-6QDW9kcIkaVTCzF7r22qZXWnC09NAT9EF48ouT0LkhQU-kTp1S2cCjS_LfcM54jMj_sXnkJDLktxww-464mo2ZzMphjiWisyA_d2f5-2GeByyd-7zwUtXNS9EerOa4SkwGcgsLbTRKctG5XkC1CJr28sL9pBOvTkBrEM6GO6hhf_LQcq-Ich7JZScsMFAWqo7qpCW62fHy7NW-wx-hMjJV-pzp4aqKPfc4b6TiUTzabbYE46UEAnlK1F-AgVBy34TV_OjwquQoRDk5Az1RHA1w9dVbZ55dP-W6ZKBunaf-BZISNUY0ZBSJyCzz7Gq4f68UqP_uT7cabFu3jGMimDwdgorHqIhdB-vunC2TQvNWhwaznPACQSH5OD2yZwxAh1WudZBHDmQg.eyJraWQiOiJBUUlCQUhnTVJ1MlZyR2FFajR3bTc2dEZqMThGMUsyVFdZbkk4WXpxS0lRamg5YTZHZ0VIWktRQXFqSU1lcjBXZE8zenM5L25BQUFBb2pDQm53WUpLb1pJaHZjTkFRY0dvSUdSTUlHT0FnRUFNSUdJQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREJvaE9GbSsvREVBbm1UOHRRSUJFSUJiRGpnSy8rM0QrcUJuUzJCbkpnajQrQ2NEWWJmRWI4L1FzNFk2VTFBdVFXTTlQblFkNW1Na2NJRWNPckRrdU1QSDdMdjQ3eW02SFVlKzd0L2VvdndYODR6YThxM1B4NGpacVdaT0g4a0ZrSkt5cDVJZVFzR2ZZVTBjWUE9PSJ9"">Link",None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

In [7]:
# Read the CSV
csv_path = "s3://apartment-pricing/preprocessed/data/csv/part-00000-0afa893a-e1f7-408c-9dc3-a22b0207eb95-c000.csv"
n_rows = 10

# Read CSV with inferSchema for better type inference (optional)
df = spark.read.csv(csv_path, header=False, inferSchema=True)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+---+---+---+---+---+------------------+---+----+
|              _c0|  _c1|_c2|_c3|_c4|_c5|_c6|               _c7|_c8| _c9|
+-----------------+-----+---+---+---+---+---+------------------+---+----+
|3249.929931640625|400.0|4.0|2.0|0.0|0.0|1.0| 8.920000076293945|6.4|23.0|
| 3873.68994140625|400.0|4.0|1.0|1.0|1.0|0.0| 7.559999942779541|9.4|93.0|
|   2741.330078125|400.0|3.0|3.0|0.0|0.0|0.0|1.7200000286102295|7.5|19.0|
| 3873.68994140625|400.0|4.0|1.0|1.0|1.0|0.0| 7.559999942779541|9.4|93.0|
|3249.929931640625|400.0|4.0|2.0|0.0|0.0|1.0| 8.920000076293945|6.4|23.0|
|   2741.330078125|400.0|3.0|3.0|0.0|0.0|0.0|1.7200000286102295|7.5|19.0|
| 3873.68994140625|400.0|4.0|1.0|1.0|1.0|0.0| 7.559999942779541|9.4|93.0|
|3249.929931640625|400.0|4.0|2.0|0.0|0.0|1.0| 8.920000076293945|6.4|23.0|
|   2741.330078125|400.0|3.0|3.0|0.0|0.0|0.0|1.7200000286102295|7.5|19.0|
|3038.699951171875|400.0|4.0|3.0|1.0|0.0|1.0| 8.869999885559082|0.4|51.0|
|   2741.330078125|400.0|3.0|3.0|0.0|0

In [5]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType
# Check for non-numeric values in 'squarefootage' column
non_numeric_squarefootage = df.filter(~col("squarefootage").cast("double").isNotNull())

# Show any rows with non-numeric values
non_numeric_squarefootage.show(truncate=False)

# Count how many rows have non-numeric values
count = non_numeric_squarefootage.count()
print(f"Number of rows with non-numeric squarefootage: {count}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------+--------+------------+----------+------------+----------+-----------------------+---------+------------+
|rent|squarefootage|numrooms|numbathrooms|hasbalcony|hasgymaccess|hasparking|neighborhoodsafetyindex|walkscore|schoolrating|
+----+-------------+--------+------------+----------+------------+----------+-----------------------+---------+------------+
+----+-------------+--------+------------+----------+------------+----------+-----------------------+---------+------------+

Number of rows with non-numeric squarefootage: 0

In [3]:
import pandas as pd
import boto3
from io import StringIO

# Input and output paths
input_path = "s3://apartment-pricing/ValidData/rental_pricing_dataset.csv"
output_path = "s3://apartment-pricing/ValidData/preprocessed/rental_pricing_dataset.csv"

# Initialize S3 client
s3 = boto3.client("s3")

# Read CSV from S3
bucket_name, key_name = input_path.replace("s3://", "").split("/", 1)
response = s3.get_object(Bucket=bucket_name, Key=key_name)
df = pd.read_csv(response["Body"])

# Transformations
df["SchoolRating"] *= 10
df["WalkScore"] /= 10

# Reorder columns to make 'rent' the first column
columns = ["Rent"] + [col for col in df.columns if col != "Rent"]
df = df[columns]

# Save the processed data back to S3
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
s3.put_object(Bucket=bucket_name, Key="ValidData/preprocessed/rental_pricing_dataset.csv", Body=csv_buffer.getvalue())

print(f"Preprocessed data saved to {output_path}")

Preprocessed data saved to s3://apartment-pricing/ValidData/preprocessed/rental_pricing_dataset.csv
